# Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews <br>

EDA: https://nycdatascience.com/blog/student-works/amazon-fine-foods-visualization/


The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use Score/Rating. A rating of 4 or 5 can be cosnidered as a positive review. A rating of 1 or 2 can be considered as negative one. A review of rating 3 is considered nuetral and such reviews are ignored from our analysis. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




# [1]. Reading Data

## [1.1] Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it is easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score is above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [2]:
# using SQLite Table to read data.
#con = sqlite3.connect('database.sqlite') 

# using the SQL Table to read data.
import mysql.connector
con=mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="Mother9922*",
    database='data1'
    )

# filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
# SELECT * FROM Reviews WHERE Score != 3 LIMIT 500000, will give top 500000 data points
# you can change the number to any other number based on your computing power

# filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000""", con) 
# for tsne assignment you can take 5k data points

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 10000""", con) 

# Give reviews with Score>3 a positive rating(1), and reviews with a score<3 a negative rating(0).
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (10000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [3]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [4]:
print(display.shape)
display.head()

(80667, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,ABXLMWJIXXAIN,B000LQOCH0,"Natalia Corres ""Natalia Corres""",1219017600,4,This is a confection that has been around a fe...,3
1,A395BORC6FGVXV,B000UA0QIQ,Karl,1307923200,2,If you are looking for the secret ingredient i...,3
2,A1UQRSCLF8GW1T,B006K2ZZ7K,"Michael D. Bigham ""M. Wassir""",1350777600,5,Great taffy at a great price. There was a wid...,2
3,A1SP2KVKFXXRU1,B006K2ZZ7K,David C. Sullivan,1340150400,5,This saltwater taffy had great flavors and was...,2
4,A1MZYO9TZK0BBI,B000E7L2R4,R. James,1322006400,5,Right now I'm mostly just sprouting this so my...,2


In [5]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
18816,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [6]:
display['COUNT(*)'].sum()

393061

#  [2] Exploratory Data Analysis

## [2.1] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [7]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As it can be seen above that same user has multiple reviews with same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [8]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [9]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(9564, 10)

In [10]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

95.64

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [11]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [12]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [13]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(9564, 10)


1    7976
0    1588
Name: Score, dtype: int64

#  [3] Preprocessing

## [3.1].  Preprocessing Review Text

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [14]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

262-TRIAL-Got a free package of these with a bottle of bloody mary mix I bought from the seller, and the advertising worked..lol,  I tried them, and shared them with 2 buddies, and we all loved them...  So im here to buy more, and noticed there were no reviews yet,...  well now there is.  They are Hot,  but not "burn your mouth forever hot"  its a nice temp...  Perfect for us.. 164
This is a very good snack that I feel great about offering my 10 month old.  She can easily  pick up the small pieces and definitely enjoys the apple cinnamon taste.  My 5 year old loves them as well!  I would definitely recommend these to anyone with small (or larger) kids.  :)
These spicy chips have a lot of great flavor but be warned, you will be left with some stanky breath afterwards! The first time I ate these was at night and then even after brushing and flossing I could taste them in my mouth the next morning. So just be careful about when you eat them, especially if you will be hanging out with peop

In [15]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

262-TRIAL-Got a free package of these with a bottle of bloody mary mix I bought from the seller, and the advertising worked..lol,  I tried them, and shared them with 2 buddies, and we all loved them...  So im here to buy more, and noticed there were no reviews yet,...  well now there is.  They are Hot,  but not "burn your mouth forever hot"  its a nice temp...  Perfect for us.. 164


In [16]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

262-TRIAL-Got a free package of these with a bottle of bloody mary mix I bought from the seller, and the advertising worked..lol,  I tried them, and shared them with 2 buddies, and we all loved them...  So im here to buy more, and noticed there were no reviews yet,...  well now there is.  They are Hot,  but not "burn your mouth forever hot"  its a nice temp...  Perfect for us.. 164
This is a very good snack that I feel great about offering my 10 month old.  She can easily  pick up the small pieces and definitely enjoys the apple cinnamon taste.  My 5 year old loves them as well!  I would definitely recommend these to anyone with small (or larger) kids.  :)
These spicy chips have a lot of great flavor but be warned, you will be left with some stanky breath afterwards! The first time I ate these was at night and then even after brushing and flossing I could taste them in my mouth the next morning. So just be careful about when you eat them, especially if you will be hanging out with peop

In [17]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [18]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

These spicy chips have a lot of great flavor but be warned, you will be left with some stanky breath afterwards! The first time I ate these was at night and then even after brushing and flossing I could taste them in my mouth the next morning. So just be careful about when you eat them, especially if you will be hanging out with people later in the day.


In [19]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

a free package of these with a bottle of bloody mary mix I bought from the seller, and the advertising worked..lol,  I tried them, and shared them with  buddies, and we all loved them...  So im here to buy more, and noticed there were no reviews yet,...  well now there is.  They are Hot,  but not "burn your mouth forever hot"  its a nice temp...  Perfect for us..


In [20]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

These spicy chips have a lot of great flavor but be warned you will be left with some stanky breath afterwards The first time I ate these was at night and then even after brushing and flossing I could taste them in my mouth the next morning So just be careful about when you eat them especially if you will be hanging out with people later in the day 


In [21]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [22]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:03<00:00, 2588.28it/s]


In [23]:
preprocessed_reviews[1500]

'spicy chips lot great flavor warned left stanky breath afterwards first time ate night even brushing flossing could taste mouth next morning careful eat especially hanging people later day'

In [24]:
final['Text']=preprocessed_reviews

## SORTING THE DATASET ACCORDING TO TIME

In [25]:
final_sorted_data=final.sort_values(by='Time')

In [26]:
print(final_sorted_data.shape)

(9564, 10)


In [27]:
final_sorted_text=final_sorted_data['Text']

# [4] Featurization

## [4.1] BAG OF WORDS

In [28]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(final_sorted_text)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

some feature names  ['aa', 'aaaa', 'aahhhs', 'ab', 'aback', 'abandon', 'abates', 'abberline', 'abbott', 'abby']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (9564, 18244)
the number of unique words  18244


## [4.2] Bi-Grams and n-Grams.

In [29]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams
# count_vect = CountVectorizer(ngram_range=(1,2))
# please do read the CountVectorizer documentation http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# you can choose these numebrs min_df=10, max_features=5000, of your choice
count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (9564, 5000)
the number of unique words including both unigrams and bigrams  5000


## [4.3] TF-IDF

In [30]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,1))
tf_idf_vect.fit(final_sorted_text)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(final_sorted_text)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['aa', 'aaaa', 'aahhhs', 'ab', 'aback', 'abandon', 'abates', 'abberline', 'abbott', 'abby']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (9564, 18244)
the number of unique words including both unigrams and bigrams  18244


## [4.4] Word2Vec

In [31]:
# Train your own Word2Vec model using your own text corpus
i=0
list_of_sentance=[]
for sentance in final_sorted_text:
    list_of_sentance.append(sentance.split())

In [32]:
# Using Google News Word2Vectors

# in this project we are using a pretrained model by google
# its 3.3G file, once you load this into your memory 
# it occupies ~9Gb, so please do this step only if you have >12G of ram
# we will provide a pickle file wich contains a dict , 
# and it contains all our courpus words as keys and  model[word] as values
# To use this code-snippet, download "GoogleNews-vectors-negative300.bin" 
# from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# it's 1.9GB in size.


# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W17SRFAzZPY
# you can comment this whole cell
# or change these varible according to your need

is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_sentance,min_count=5,size=300, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")

[('excellent', 0.9105055332183838), ('good', 0.8750794529914856), ('alternative', 0.8680955767631531), ('regular', 0.8542636632919312), ('looking', 0.8356977701187134), ('overall', 0.8344712853431702), ('value', 0.8298949003219604), ('doritos', 0.820156991481781), ('tasty', 0.8198351860046387), ('exactly', 0.8156177401542664)]
[('varieties', 0.9928370714187622), ('absolute', 0.9927341938018799), ('hands', 0.9906676411628723), ('smoothest', 0.9891992807388306), ('amongst', 0.9886382222175598), ('gourmet', 0.9873467087745667), ('names', 0.9870598912239075), ('salsas', 0.9864417314529419), ('britt', 0.986007809638977), ('mediocre', 0.985862672328949)]


In [33]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  5652
sample words  ['really', 'good', 'idea', 'final', 'product', 'outstanding', 'use', 'car', 'window', 'everybody', 'asks', 'bought', 'made', 'two', 'thumbs', 'received', 'shipment', 'could', 'hardly', 'wait', 'try', 'love', 'call', 'instead', 'stickers', 'removed', 'easily', 'daughter', 'designed', 'printed', 'windows', 'beautifully', 'print', 'shop', 'program', 'going', 'lot', 'fun', 'everywhere', 'like', 'tv', 'computer', 'winter', 'fresh', 'blueberries', 'exceed', 'food', 'budget', 'dried', 'blue']


## [4.4.1] Converting text into vectors using Avg W2V, TFIDF-W2V

#### [4.4.1.1] Avg W2v

In [34]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance): # for each review/sentence
    sent_vec = np.zeros(300) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|█████████████████████████████████████████████████████████████████████████████| 9564/9564 [00:11<00:00, 830.19it/s]

9564
300


#### [4.4.1.2] TFIDF weighted W2v

In [35]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
tf_idf_matrix = model.fit_transform(final_sorted_text)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [36]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_of_sentance): # for each review/sentence 
    sent_vec = np.zeros(300) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|█████████████████████████████████████████████████████████████████████████████| 9564/9564 [01:18<00:00, 121.40it/s]


# [5] Assignment 3: KNN

<ol>
    <li><strong>Apply Knn(brute force version) on these feature sets</strong>
        <ul>
            <li><font color='red'>SET 1:</font>Review text, preprocessed one converted into vectors using (BOW)</li>
            <li><font color='red'>SET 2:</font>Review text, preprocessed one converted into vectors using (TFIDF)</li>
            <li><font color='red'>SET 3:</font>Review text, preprocessed one converted into vectors using (AVG W2v)</li>
            <li><font color='red'>SET 4:</font>Review text, preprocessed one converted into vectors using (TFIDF W2v)</li>
        </ul>
    </li>
    <br>
    <li><strong>Apply Knn(kd tree version) on these feature sets</strong>
        <br><font color='red'>NOTE: </font>sklearn implementation of kd-tree accepts only dense matrices, you need to convert the sparse matrices of CountVectorizer/TfidfVectorizer into dense matices. You can convert sparse matrices to dense using .toarray() attribute. For more information please visit this <a href='https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.toarray.html'>link</a>
        <ul>
            <li><font color='red'>SET 5:</font>Review text, preprocessed one converted into vectors using (BOW) but with restriction on maximum features generated.
            <pre>
            count_vect = CountVectorizer(min_df=10, max_features=500) 
            count_vect.fit(preprocessed_reviews)
            </pre>
            </li>
            <li><font color='red'>SET 6:</font>Review text, preprocessed one converted into vectors using (TFIDF) but with restriction on maximum features generated.
            <pre>
                tf_idf_vect = TfidfVectorizer(min_df=10, max_features=500)
                tf_idf_vect.fit(preprocessed_reviews)
            </pre>
            </li>
            <li><font color='red'>SET 3:</font>Review text, preprocessed one converted into vectors using (AVG W2v)</li>
            <li><font color='red'>SET 4:</font>Review text, preprocessed one converted into vectors using (TFIDF W2v)</li>
        </ul>
    </li>
    <br>
    <li><strong>The hyper paramter tuning(find best K)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>Find the best hyper paramter using k-fold cross validation or simple cross validation data</li>
    <li>Use gridsearch cv or randomsearch cv or you can also write your own for loops to do this task of hyperparameter tuning</li>
        </ul>
    </li>
    <br>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='train_cv_auc.JPG' width=300px></li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='train_test_auc.JPG' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='confusion_matrix.png' width=300px></li>
        </ul>
    </li>
    <br>
    <li><strong>Conclusion</strong>
        <ul>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format. To print out a table please refer to this prettytable library<a href='http://zetcode.com/python/prettytable/'> link</a> 
        <img src='summary.JPG' width=400px>
    </li>
        </ul>
</ol>

<h4><font color='red'>Note: Data Leakage</font></h4>

1. There will be an issue of data-leakage if you vectorize the entire data and then split it into train/cv/test.
2. To avoid the issue of data-leakag, make sure to split your data first and then vectorize it. 
3. While vectorizing your data, apply the method fit_transform() on you train data, and apply the method transform() on cv/test data.
4. For more details please go through this <a href='https://soundcloud.com/applied-ai-course/leakage-bow-and-tfidf'>link.</a>

## [5.1] Applying KNN brute force

### [5.1.1] Applying KNN brute force on BOW,<font color='red'> SET 1</font>

In [37]:
print(final_counts.shape)
print(type(final_counts))

(9564, 18244)
<class 'scipy.sparse.csr.csr_matrix'>


In [38]:
X=final_counts
Y=final_sorted_data['Score']

# 5.1.1.1. Splitting into train, cv and test (Simple Cross Validation)


In [39]:
# from sklearn.model_selection import train_test_split
### This will not work because train_test_split() splits data randomly. What we want is a time-based splitting on
### the dataset that we have sorted chronologically
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [40]:
print(X.shape)
print(Y.shape)
print(type(Y))

(9564, 18244)
(9564,)
<class 'pandas.core.series.Series'>


In [41]:
#function to split dataset into train and test datasets

def train_test_splitter(X, y, test_size):
    train_size = 1 - test_size
    train_row_upper_index = round(train_size*X.shape[0])
    test_row_lower_index = train_row_upper_index + 1
    
    #print(train_row_upper_index)
    
    X_train = X[:train_row_upper_index + 1, :]
    X_test = X[test_row_lower_index:, :]
    y_train = y.iloc[:train_row_upper_index + 1]
    y_test = y.iloc[test_row_lower_index:]
    
    return X_train, X_test, y_train, y_test

In [42]:
X_train, X_test, Y_train, Y_test = train_test_splitter(X, Y, test_size = 0.20)

In [43]:
print(type(X_train))
print(X_train.shape[0] + X_test.shape[0])

<class 'scipy.sparse.csr.csr_matrix'>
9564


In [44]:
# Splitting the previous X_train and y_train into X_train, X_cv and y_train, y_cv
X_train, X_cv, Y_train, Y_cv = train_test_splitter(X_train, Y_train, 0.20)

# 5.1.1.2. Training and Testing

In [45]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [46]:
# KNN Classifier and cross validator using Simple Cross Validation

def knn_trainer_and_cross_validator(k, X_train, y_train, X_cv, y_cv):
    
    # Side note: note that X_train and y_train are sparse matrices, and not numpy arrays
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)
    
    y_pred_cv = knn.predict(X_cv)
    
    accuracy = accuracy_score(y_cv, y_pred_cv) * 100
    
    return accuracy

In [47]:
accuracies_for_diff_k = []
for i in range(1, 100):
    accuracy = knn_trainer_and_cross_validator(i, X_train, Y_train, X_cv, Y_cv)
    accuracies_for_diff_k.append(accuracy)
    print('Accuracy for k = ' + str(i) + ' is ' + str(accuracy))

Accuracy for k = 1 is 51.73315892740354
Accuracy for k = 2 is 37.60627861347286
Accuracy for k = 3 is 42.51144538914323
Accuracy for k = 4 is 74.55853499018966
Accuracy for k = 5 is 79.79071288423808
Accuracy for k = 6 is 78.67887508175278
Accuracy for k = 7 is 80.57553956834532
Accuracy for k = 8 is 80.2485284499673
Accuracy for k = 9 is 80.96795291039895
Accuracy for k = 10 is 80.90255068672334
Accuracy for k = 11 is 80.90255068672334
Accuracy for k = 12 is 80.90255068672334
Accuracy for k = 13 is 80.96795291039895
Accuracy for k = 14 is 80.96795291039895
Accuracy for k = 15 is 80.96795291039895
Accuracy for k = 16 is 80.96795291039895
Accuracy for k = 17 is 80.96795291039895
Accuracy for k = 18 is 80.96795291039895
Accuracy for k = 19 is 80.96795291039895
Accuracy for k = 20 is 80.96795291039895
Accuracy for k = 21 is 80.96795291039895
Accuracy for k = 22 is 80.96795291039895
Accuracy for k = 23 is 80.96795291039895
Accuracy for k = 24 is 80.96795291039895
Accuracy for k = 25 is 80.

In [48]:
print(max(accuracies_for_diff_k), accuracies_for_diff_k.index(max(accuracies_for_diff_k)), sep = '\t')
k_value=(accuracies_for_diff_k.index(max(accuracies_for_diff_k))+1)

80.96795291039895	8


# for k=9 accuracy is maximum

In [49]:
knn = KNeighborsClassifier(n_neighbors = k_value)

In [50]:
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=9)

In [51]:
### finally testing after tuning the hyperparameter 'k' on the cross validation set
y_pred_test = knn.predict(X_test)

In [52]:
accuracy_score(Y_test,y_pred_test)*100

81.85146443514645

### [5.1.2] Applying KNN brute force on TFIDF,<font color='red'> SET 2</font>

In [53]:
print(final_tf_idf.shape)
print(type(final_tf_idf))

(9564, 18244)
<class 'scipy.sparse.csr.csr_matrix'>


In [54]:
X=final_tf_idf
Y=final_sorted_data['Score']

# 5.1.2.1. Splitting into train, cv and test (Simple Cross Validation)


In [55]:
# from sklearn.model_selection import train_test_split
### This will not work because train_test_split() splits data randomly. What we want is a time-based splitting on
### the dataset that we have sorted chronologically
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [56]:
print(X.shape)
print(Y.shape)
print(type(Y))

(9564, 18244)
(9564,)
<class 'pandas.core.series.Series'>


In [57]:
#function to split dataset into train and test datasets

def train_test_splitter(X, y, test_size):
    train_size = 1 - test_size
    train_row_upper_index = round(train_size*X.shape[0])
    test_row_lower_index = train_row_upper_index + 1
    
    #print(train_row_upper_index)
    
    X_train = X[:train_row_upper_index + 1, :]
    X_test = X[test_row_lower_index:, :]
    y_train = y.iloc[:train_row_upper_index + 1]
    y_test = y.iloc[test_row_lower_index:]
    
    return X_train, X_test, y_train, y_test

In [58]:
#splitting data into train and test set
X_train, X_test, Y_train, Y_test = train_test_splitter(X, Y, test_size = 0.20)

In [59]:
print(type(X_train))
print(X_train.shape[0] + X_test.shape[0])

<class 'scipy.sparse.csr.csr_matrix'>
9564


In [60]:
# Splitting the previous X_train and Y_train into X_train, X_cv and Y_train, Y_cv
X_train, X_cv, Y_train, Y_cv = train_test_splitter(X_train, Y_train, 0.20)

# 5.1.2.2. Training and Testing

In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [62]:
# KNN Classifier and cross validator using Simple Cross Validation

def knn_trainer_and_cross_validator(k, X_train, y_train, X_cv, y_cv):
    
    # Side note: note that X_train and y_train are sparse matrices, and not numpy arrays
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)
    
    y_pred_cv = knn.predict(X_cv)
    
    accuracy = accuracy_score(y_cv, y_pred_cv) * 100
    
    return accuracy

In [63]:
accuracies_for_diff_k = []
for i in range(1, 100):
    accuracy = knn_trainer_and_cross_validator(i, X_train, Y_train, X_cv, Y_cv)
    accuracies_for_diff_k.append(accuracy)
    print('Accuracy for k = ' + str(i) + ' is ' + str(accuracy))

Accuracy for k = 1 is 81.09875735775016
Accuracy for k = 2 is 81.09875735775016
Accuracy for k = 3 is 80.96795291039895
Accuracy for k = 4 is 80.96795291039895
Accuracy for k = 5 is 80.96795291039895
Accuracy for k = 6 is 80.96795291039895
Accuracy for k = 7 is 80.96795291039895
Accuracy for k = 8 is 80.96795291039895
Accuracy for k = 9 is 80.96795291039895
Accuracy for k = 10 is 80.96795291039895
Accuracy for k = 11 is 80.96795291039895
Accuracy for k = 12 is 80.96795291039895
Accuracy for k = 13 is 80.96795291039895
Accuracy for k = 14 is 80.96795291039895
Accuracy for k = 15 is 80.96795291039895
Accuracy for k = 16 is 80.96795291039895
Accuracy for k = 17 is 80.96795291039895
Accuracy for k = 18 is 80.96795291039895
Accuracy for k = 19 is 80.96795291039895
Accuracy for k = 20 is 80.96795291039895
Accuracy for k = 21 is 80.96795291039895
Accuracy for k = 22 is 80.96795291039895
Accuracy for k = 23 is 80.96795291039895
Accuracy for k = 24 is 80.96795291039895
Accuracy for k = 25 is 80

In [64]:
print(max(accuracies_for_diff_k), accuracies_for_diff_k.index(max(accuracies_for_diff_k)), sep = '\t')
k_value=(accuracies_for_diff_k.index(max(accuracies_for_diff_k))+1)

81.09875735775016	0


# from above k=2 gives the best accuracy
# so we choose k=2 Since the cross-validation accuracy obtained for k = 2 to above are same and less than k=1,2

In [65]:
### index = 1 has the maximum accuracy. This index corresponds to k = 2 (from the above printed list of accuracies)
knn = KNeighborsClassifier(n_neighbors = k_value)

In [66]:
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=1)

In [67]:
### finally testing after tuning the hyperparameter 'k' on the cross validation set
y_pred_test = knn.predict(X_test)

In [68]:
accuracy_score(Y_test,y_pred_test)*100

82.11297071129707

### [5.1.3] Applying KNN brute force on AVG W2V,<font color='red'> SET 3</font>

In [69]:
len(sent_vectors)

9564

In [70]:
X=np.array(sent_vectors)
Y=final_sorted_data['Score']

In [71]:
print(X.shape)
print(Y.shape)

(9564, 300)
(9564,)


In [72]:
#function to split dataset into train and test datasets

def train_test_splitter(X, y, test_size):
    train_size = 1 - test_size
    train_row_upper_index = round(train_size*X.shape[0])
    test_row_lower_index = train_row_upper_index + 1
    
    #print(train_row_upper_index)
    
    X_train = X[:train_row_upper_index + 1, :]
    X_test = X[test_row_lower_index:, :]
    y_train = y.iloc[:train_row_upper_index + 1]
    y_test = y.iloc[test_row_lower_index:]
    
    return X_train, X_test, y_train, y_test

In [73]:
X_train, X_test, Y_train, Y_test = train_test_splitter(X, Y, test_size = 0.20)

In [74]:
print(type(X_train))
print(X_train.shape[0] + X_test.shape[0])

<class 'numpy.ndarray'>
9564


In [75]:
# Splitting the previous X_train and y_train into X_train, X_cv and y_train, y_cv
X_train, X_cv, Y_train, Y_cv = train_test_splitter(X_train, Y_train, 0.20)

# 5.1.1.2. Training and Testing

In [76]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [77]:
# KNN Classifier and cross validator using Simple Cross Validation

def knn_trainer_and_cross_validator(k, X_train, y_train, X_cv, y_cv):
    
    # Side note: note that X_train and y_train are sparse matrices, and not numpy arrays
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)
    
    y_pred_cv = knn.predict(X_cv)
    
    accuracy = accuracy_score(y_cv, y_pred_cv) * 100
    
    return accuracy

In [78]:
accuracies_for_diff_k = []
for i in range(1, 100):
    accuracy = knn_trainer_and_cross_validator(i, X_train, Y_train, X_cv, Y_cv)
    accuracies_for_diff_k.append(accuracy)
    print('Accuracy for k = ' + str(i) + ' is ' + str(accuracy))

Accuracy for k = 1 is 74.75474166121649
Accuracy for k = 2 is 69.91497710922171
Accuracy for k = 3 is 77.82864617396991
Accuracy for k = 4 is 77.24002616088947
Accuracy for k = 5 is 80.3793328973185
Accuracy for k = 6 is 79.59450621321125
Accuracy for k = 7 is 80.3793328973185
Accuracy for k = 8 is 79.98691955526488
Accuracy for k = 9 is 80.44473512099411
Accuracy for k = 10 is 80.05232177894048
Accuracy for k = 11 is 80.44473512099411
Accuracy for k = 12 is 80.31393067364291
Accuracy for k = 13 is 80.77174623937215
Accuracy for k = 14 is 80.51013734466972
Accuracy for k = 15 is 80.83714846304775
Accuracy for k = 16 is 80.90255068672334
Accuracy for k = 17 is 81.22956180510137
Accuracy for k = 18 is 80.83714846304775
Accuracy for k = 19 is 81.16415958142576
Accuracy for k = 20 is 81.03335513407455
Accuracy for k = 21 is 80.83714846304775
Accuracy for k = 22 is 80.77174623937215
Accuracy for k = 23 is 81.09875735775016
Accuracy for k = 24 is 81.03335513407455
Accuracy for k = 25 is 81.1

In [79]:
print(max(accuracies_for_diff_k), accuracies_for_diff_k.index(max(accuracies_for_diff_k)), sep = '\t')
k_value=(accuracies_for_diff_k.index(max(accuracies_for_diff_k))+1)

81.3603662524526	42


# from above k=18 gives the best accuracy
# so we choose k=18 Since the cross-validation accuracy obtained is maxm

In [80]:
### index = 17 has the maximum accuracy. This index corresponds to k = 18 (from the above printed list of accuracies)
knn = KNeighborsClassifier(n_neighbors = k_value)

In [81]:
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=43)

In [82]:
### finally testing after tuning the hyperparameter 'k' on the cross validation set
y_pred_test = knn.predict(X_test)

In [83]:
accuracy_score(Y_test,y_pred_test)*100

82.06066945606695

### [5.1.4] Applying KNN brute force on TFIDF W2V,<font color='red'> SET 4</font>

In [84]:
len(tfidf_sent_vectors)

9564

In [85]:
X=np.array(tfidf_sent_vectors)
Y=final_sorted_data['Score']

In [86]:
print(X.shape)
print(Y.shape)

(9564, 300)
(9564,)


In [87]:
#function to split dataset into train and test datasets

def train_test_splitter(X, y, test_size):
    train_size = 1 - test_size
    train_row_upper_index = round(train_size*X.shape[0])
    test_row_lower_index = train_row_upper_index + 1
    
    #print(train_row_upper_index)
    
    X_train = X[:train_row_upper_index + 1, :]
    X_test = X[test_row_lower_index:, :]
    y_train = y.iloc[:train_row_upper_index + 1]
    y_test = y.iloc[test_row_lower_index:]
    
    return X_train, X_test, y_train, y_test

In [88]:
X_train, X_test, Y_train, Y_test = train_test_splitter(X, Y, test_size = 0.20)

In [89]:
print(type(X_train))
print(X_train.shape[0] + X_test.shape[0])

<class 'numpy.ndarray'>
9564


In [90]:
# Splitting the previous X_train and y_train into X_train, X_cv and y_train, y_cv
X_train, X_cv, Y_train, Y_cv = train_test_splitter(X_train, Y_train, 0.20)

# 5.1.1.2. Training and Testing

In [91]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [92]:
# KNN Classifier and cross validator using Simple Cross Validation

def knn_trainer_and_cross_validator(k, X_train, y_train, X_cv, y_cv):
    
    # Side note: note that X_train and y_train are sparse matrices, and not numpy arrays
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)
    
    y_pred_cv = knn.predict(X_cv)
    
    accuracy = accuracy_score(y_cv, y_pred_cv) * 100
    
    return accuracy

In [93]:
accuracies_for_diff_k = []
for i in range(1, 100):
    accuracy = knn_trainer_and_cross_validator(i, X_train, Y_train, X_cv, Y_cv)
    accuracies_for_diff_k.append(accuracy)
    print('Accuracy for k = ' + str(i) + ' is ' + str(accuracy))

Accuracy for k = 1 is 75.21255722694572
Accuracy for k = 2 is 67.49509483322433
Accuracy for k = 3 is 77.95945062132112
Accuracy for k = 4 is 75.86657946370177
Accuracy for k = 5 is 80.31393067364291
Accuracy for k = 6 is 79.13669064748201
Accuracy for k = 7 is 80.44473512099411
Accuracy for k = 8 is 79.65990843688685
Accuracy for k = 9 is 80.57553956834532
Accuracy for k = 10 is 80.2485284499673
Accuracy for k = 11 is 80.90255068672334
Accuracy for k = 12 is 80.77174623937215
Accuracy for k = 13 is 80.96795291039895
Accuracy for k = 14 is 80.83714846304775
Accuracy for k = 15 is 80.90255068672334
Accuracy for k = 16 is 81.22956180510137
Accuracy for k = 17 is 81.03335513407455
Accuracy for k = 18 is 81.16415958142576
Accuracy for k = 19 is 81.16415958142576
Accuracy for k = 20 is 81.22956180510137
Accuracy for k = 21 is 81.16415958142576
Accuracy for k = 22 is 80.96795291039895
Accuracy for k = 23 is 81.03335513407455
Accuracy for k = 24 is 81.09875735775016
Accuracy for k = 25 is 81.

In [94]:
print(max(accuracies_for_diff_k), accuracies_for_diff_k.index(max(accuracies_for_diff_k)), sep = '\t')
k_value=(accuracies_for_diff_k.index(max(accuracies_for_diff_k))+1)

81.29496402877699	35


# from above k=18 gives the best accuracy

In [95]:
### index = 17 has the maximum accuracy. This index corresponds to k = 18 (from the above printed list of accuracies)
knn = KNeighborsClassifier(n_neighbors = k_value)

In [96]:
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=36)

In [97]:
### finally testing after tuning the hyperparameter 'k' on the cross validation set
y_pred_test = knn.predict(X_test)

In [98]:
accuracy_score(Y_test,y_pred_test)*100

81.9560669456067

## [5.2] Applying KNN kd-tree

### [5.2.1] Applying KNN kd-tree on BOW,<font color='red'> SET 5</font>

In [99]:
# Please write all the code with proper documentation

### [5.2.2] Applying KNN kd-tree on TFIDF,<font color='red'> SET 6</font>

In [100]:
# Please write all the code with proper documentation

### [5.2.3] Applying KNN kd-tree on AVG W2V,<font color='red'> SET 3</font>

In [101]:
# Please write all the code with proper documentation

### [5.2.4] Applying KNN kd-tree on TFIDF W2V,<font color='red'> SET 4</font>

In [102]:
# Please write all the code with proper documentation

# [6] Conclusions

In [103]:
# Please compare all your models using Prettytable library